<a href="https://colab.research.google.com/github/Malik-Raheel/data_analysis/blob/main/Low_Freq_Tradign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install ccxt pandas numpy

In [ ]:


import ccxt
import pandas as pd
import numpy as np
from datetime import datetime

# Initialize the CCXT exchange object (kucoin in this case)
exchange = ccxt.kucoin({
    'rateLimit': 1200,
    'enableRateLimit': True,
})

# Function to fetch OHLCV data
def fetch_data(symbol, timeframe='15m', limit=200):
    """Fetch historical OHLCV data for a given symbol and timeframe."""
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

# Function to calculate RSI
def calculate_rsi(df, window=14):
    """Calculate Relative Strength Index (RSI)."""
    delta = df['close'].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=window).mean()
    avg_loss = pd.Series(loss).rolling(window=window).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

# Function to calculate EMA
def calculate_ema(df, window):
    """Calculate Exponential Moving Average (EMA)."""
    df[f'EMA_{window}'] = df['close'].ewm(span=window, adjust=False).mean()
    return df

# Function to find trading signals
def find_signals(df):
    """Find buy/sell signals based on the strategy."""
    signals = []
    for i in range(len(df)):
        # Conditions for a buy signal
        if (
            df['RSI'].iloc[i] < 35 and  # RSI below 35 (oversold)
            df['EMA_15'].iloc[i] > df['EMA_50'].iloc[i] and  # EMA 15 > EMA 50
            df['volume'].iloc[i] > 1.3 * df['volume'].rolling(window=20).mean().iloc[i]  # Volume spike
        ):
            signals.append("BUY")
        else:
            signals.append("HOLD")
    df['Signal'] = signals
    return df

# Main function
def main():
    # List of assets to trade
    assets = ['BTC/USDT', 'ETH/USDT', 'AVAX/USDT', 'XRP/USDT', 'ADA/USDT',
              'CRV/USDT', 'SOL/USDT', 'LINK/USDT', 'DOGE/USDT', 'INJ/USDT']

    # Timeframe for signals
    timeframe = '15m'

    # Loop through each asset
    for asset in assets:
        print(f"\nFetching data for {asset}...")
        df = fetch_data(asset, timeframe)

        # Add technical indicators
        df = calculate_rsi(df)
        df = calculate_ema(df, window=15)
        df = calculate_ema(df, window=50)

        # Find trading signals
        df = find_signals(df)

        # Filter for recent BUY signals
        recent_signals = df[df['Signal'] == 'BUY'][['timestamp', 'close', 'RSI', 'EMA_15', 'EMA_50', 'volume', 'Signal']].tail(1)

        if not recent_signals.empty:
            print(f"Recent Signal for {asset}:")
            print(recent_signals)
        else:
            print(f"No recent BUY signals for {asset}.")

if __name__ == "__main__":
    main()
